## 1. Import necessary module

In [64]:
from scipy.optimize import milp, LinearConstraint, Bounds
from scipy.sparse import csc_matrix
from qiskit_optimization import QuadraticProgram
from qiskit import QiskitError
import numpy as np
from qiskit_optimization.algorithms import GurobiOptimizer


In [2]:
%load_ext autoreload
%autoreload 2

## 2. Create a demo problem

### Example 1


$${\rm max} \ x_1+2x_2+2x_3+5x_4 \\

-x_1+x_2+x_3+x_4 \leq 1 \\ 3x_1+2x_2 - x_3+x_4 \leq 12 \\ x_1+x_2 +x_3+x_4 \leq 10 \\ 2x_1+3x_2 + 3x_3+x_4 \geq 12 \\ 

x_1, x_2 \in \mathcal{Z}\\
x_4 \in \{0, 1\}
$$

Let's solve it with the milp solver!

In [24]:
## constant value in objective

c = -np.array([1, 2, 1, 5])  
A = np.array([[-1, 1, 1, 1], [3, 2, -1, 1], [1, 1, 1, 1], [2, 3, 3, 1]])
b_u = np.array([1, 12, 10, np.inf])
b_l = np.array([-np.inf, -np.inf, -np.inf, 12])

constraints = LinearConstraint(A, b_l, b_u)
integrality = np.array([1, 1, 0, 1])
bounds = Bounds([0, 0, 0, 0],[np.inf, np.inf, np.inf, 1])
#integrality = np.zeros(4)
res = milp(c=c, constraints=constraints, integrality=integrality, bounds=bounds)

print(res)

            fun: -14.0
        message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
 mip_dual_bound: -14.0
        mip_gap: 0.0
 mip_node_count: 1
         status: 0
        success: True
              x: array([4., 1., 3., 1.])


We see the MILP solver tells us the object function is maximized when $x =  [4, 1, 3, 1]$.

As the next step, let's create a corresponding `QuadraticProgram` class in `qiskit_optimization`.

In [22]:
## Consider the possibility of using sparse matrix(to_dict)
## 1e20=inf
qp = QuadraticProgram('example-1')
qp.integer_var(name='x1')
qp.integer_var(name='x2')
qp.continuous_var(name='x3')
qp.binary_var(name='x4')
#qp.binary_var_list(2000)
qp.maximize(linear={'x1': 1, 'x2': 2, 'x3': 1, 'x4': 5})
#qp.minimize(linear={'x2': 1})
qp.linear_constraint({'x1': -1, 'x2': 1, 'x3': 1, 'x4': 1},'<=',1)
qp.linear_constraint({'x1': 1, 'x2': 1, 'x3': 1, 'x4': 1},'<=',10)
qp.linear_constraint({'x1': 3, 'x2': 2, 'x3': -1, 'x4': 1},'<=',12)
qp.linear_constraint({'x1': 2, 'x2': 3, 'x3': 3, 'x4': 1},'>=',12)
qp.linear_cons
print(qp.prettyprint())

Problem name: example-1

Maximize
  x1 + 2*x2 + x3 + 5*x4

Subject to
  Linear constraints (4)
    -x1 + x2 + x3 + x4 <= 1  'c0'
    x1 + x2 + x3 + x4 <= 10  'c1'
    3*x1 + 2*x2 - x3 + x4 <= 12  'c2'
    2*x1 + 3*x2 + 3*x3 + x4 >= 12  'c3'

  Integer variables (2)
    0 <= x1
    0 <= x2

  Continuous variables (1)
    0 <= x3

  Binary variables (1)
    x4



We solve the `qp(QuadraticProgram)` with the Gurobi solver.

In [23]:
gurobi_result = GurobiOptimizer().solve(qp)
print(gurobi_result.prettyprint())

objective function value: 14.0
variable values: x1=5.0, x2=0.0, x3=4.0, x4=1.0
status: SUCCESS


In [61]:
dir(gurobi_result)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_fval',
 '_raw_results',
 '_samples',
 '_status',
 '_variable_names',
 '_variables',
 '_variables_dict',
 '_x',
 'fval',
 'get_correlations',
 'prettyprint',
 'raw_results',
 'samples',
 'status',
 'variable_names',
 'variables',
 'variables_dict',
 'x']

## 3. Build a Converter

The converter should be able to extract necessary information from the `QuadraticProblem` class and input it to the milp solver.

In [58]:
qp.variables[3].lowerbound

0

In [90]:
## First, Check if qp is linear
## Check objective is linear
##from qiskit_optimization import vartype

if qp.objective.quadratic.to_dict()!={}:
    raise QiskitError('Obejective function is not linear!')

## Check constrains are linear
if qp.quadratic_constraints:
    raise QiskitError('Constraints are not linear!')

## Sense: 1 for minimization and -1 for maximization 
sense = qp.objective.sense.value

## cost function for milp solver
c_qp = qp.objective.linear.to_array() * sense  

## constraints for milp solver
for i, constraint in enumerate(qp.linear_constraints):

    constraint_array = constraint.linear.to_array()
    constraint_sense = constraint.sense.value # 0 for leq, 1 for geq
    constraint_value = constraint.rhs
    if i==0:
        A_qp = [constraint_array]
        bl_qp = np.array([constraint_value if constraint_sense==1 else -np.inf])
        bu_qp = np.array([constraint_value if constraint_sense==0 else np.inf])
    else:
        A_qp.append(constraint_array)  ## Not efficient. Use list directly.
        bl_qp = np.append(bl_qp, [constraint_value if constraint_sense==1 else -np.inf])
        bu_qp = np.append(bu_qp, [constraint_value if constraint_sense==0 else np.inf])

constraints = LinearConstraint(A_qp, bl_qp, bu_qp)

## integrity for milp solver
## Use Enum and VarType to compare
integrality = np.array([ 1 if variable.vartype.value==2 or variable.vartype.value==1 else 0 for variable in qp.variables]) ## check on other vartype!
lower_bounds = np.array([var.lowerbound for var in qp.variables])
upper_bounds = np.array([var.upperbound for var in qp.variables])
bounds = Bounds(lower_bounds,upper_bounds)
raw_res = milp(c=c_qp, constraints=constraints, integrality=integrality, bounds=bounds)

print(raw_res)



            fun: -14.0
        message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
 mip_dual_bound: -14.0
        mip_gap: 0.0
 mip_node_count: 1
         status: 0
        success: True
              x: array([4., 1., 3., 1.])


In [88]:
## Sparse converter based on to_dict

if qp.objective.quadratic.to_dict()!={}:
    raise QiskitError('Obejective function is not linear!')

## Check constrains are linear
if qp.quadratic_constraints:
    raise QiskitError('Constraints are not linear!')

## Sense: 1 for minimization and -1 for maximization 
sense = qp.objective.sense.value

## cost function for milp solver
c_qp = qp.objective.linear.to_array() * sense  


A_qp = csc_matrix((len(qp.linear_constraints), len(qp.variables)))
bl_qp = np.array([])
bu_qp = np.array([])
## constraints for milp solver
for i, constraint in enumerate(qp.linear_constraints):
    constraint_dict = constraint.linear.to_dict()
    constraint_sense = constraint.sense.value # 0 for leq, 1 for geq
    constraint_value = constraint.rhs
    
    for var in constraint_dict:
        A_qp[i,var]= constraint_dict[var]

    bl_qp = np.append(bl_qp, [constraint_value if constraint_sense==1 else -np.inf])
    #print(bl_qp)
    bu_qp = np.append(bu_qp, [constraint_value if constraint_sense==0 else np.inf])

constraints = LinearConstraint(A_qp, bl_qp, bu_qp)


## integrity for milp solver
## Use Enum and VarType to compare
integrality = np.array([ 1 if variable.vartype.value==2 or variable.vartype.value==1 else 0 for variable in qp.variables]) ## check on other vartype!
lower_bounds = np.array([var.lowerbound for var in qp.variables])
upper_bounds = np.array([var.upperbound for var in qp.variables])
bounds = Bounds(lower_bounds,upper_bounds)
raw_res = milp(c=c_qp, constraints=constraints, integrality=integrality, bounds=bounds)

print(raw_res)

            fun: -14.0
        message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
 mip_dual_bound: -14.0
        mip_gap: 0.0
 mip_node_count: 1
         status: 0
        success: True
              x: array([4., 1., 3., 1.])


/Users/dranz/Library/Python/3.9/lib/python/site-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


In [87]:
A_qp.toarray()

array([[-1.,  1.,  3.,  2.],
       [ 1.,  1.,  2.,  3.],
       [ 1.,  1., -1.,  3.],
       [ 1.,  1.,  1.,  1.]])

In [65]:
A = csc_matrix((4,4))

In [67]:
A[0,0]

0.0

In [29]:
A = qp.get_linear_constraint(0).linear.to_array()
B = qp.get_linear_constraint(1).linear.to_array()
A = np.vstack((A, B))
A

array([[-1.,  1.,  1.],
       [ 3.,  2., -1.]])

In [62]:
qp.get_linear_constraint(0).sense

<ConstraintSense.LE: 0>

In [26]:
integrality = np.array([ 1 if variable.vartype.value==2 else 0 for variable in qp.variables])